# Turning multi-class classification into Binary classification
## (0) for low severity and (1) for high severity

In [2]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import precision_recall_curve, auc
import xgboost as xgb
import re
from collections import Counter
from sklearn.model_selection import GridSearchCV



# Load the dataset
file_path = 'Resources/US_Accidents_2022_Phoenix_Metro2.csv'
df = pd.read_csv(file_path, low_memory=False)

# Drop columns with too many missing values
df.drop(columns=['End_Lat', 'End_Lng'], inplace=True)

# Handle missing values in other columns
df.dropna(inplace=True)

# Convert date columns to datetime and then to numerical values (e.g., ordinal)
df['Accident_Date'] = pd.to_datetime(df['Accident_Date']).map(pd.Timestamp.toordinal)

# Create labels and features
y = df['Severity']
X = df.drop(columns=['Severity'])

# Identify non-numeric columns
non_numeric_columns = X.select_dtypes(include=['object']).columns

# One-hot encode categorical columns
X = pd.get_dummies(X, columns=non_numeric_columns, drop_first=True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Instantiate and fit the Logistic Regression model
clf = LogisticRegression(random_state=1, max_iter=1000)
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Evaluate the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Combine classes into binary categories
df['binary_target'] = df['Severity'].apply(lambda x: 0 if x in [1, 2] else 1)

# Features and new binary target
X = df.drop(columns=['Severity', 'binary_target'])
y = df['binary_target']

# One-hot encode categorical columns again
X = pd.get_dummies(X, columns=non_numeric_columns, drop_first=True)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train the logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Confusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", class_report)


[[  0   2   0   0]
 [  0 174   0   0]
 [  0  11   0   0]
 [  0   2   0   0]]
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         2
           2       0.92      1.00      0.96       174
           3       0.00      0.00      0.00        11
           4       0.00      0.00      0.00         2

    accuracy                           0.92       189
   macro avg       0.23      0.25      0.24       189
weighted avg       0.85      0.92      0.88       189

Confusion Matrix:
 [[210   1]
 [ 16   0]]

Classification Report:
               precision    recall  f1-score   support

           0       0.93      1.00      0.96       211
           1       0.00      0.00      0.00        16

    accuracy                           0.93       227
   macro avg       0.46      0.50      0.48       227
weighted avg       0.86      0.93      0.89       227



# Introducing SMOTE to balance out classes

In [4]:
# Load the dataset
file_path = 'Resources/US_Accidents_2022_Phoenix_Metro2.csv'
df = pd.read_csv(file_path, low_memory=False)

# Drop columns with too many missing values
df.drop(columns=['End_Lat', 'End_Lng'], inplace=True)

# Handle missing values in other columns
df.dropna(inplace=True)

# Convert date columns to datetime and then to numerical values (e.g., ordinal)
df['Accident_Date'] = pd.to_datetime(df['Accident_Date']).map(pd.Timestamp.toordinal)

# Combine classes into binary categories
df['binary_target'] = df['Severity'].apply(lambda x: 0 if x in [1, 2] else 1)

# Create labels and features
y = df['binary_target']
X = df.drop(columns=['Severity', 'binary_target'])

# Identify non-numeric columns
non_numeric_columns = X.select_dtypes(include=['object']).columns

# One-hot encode categorical columns
X = pd.get_dummies(X, columns=non_numeric_columns, drop_first=True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Apply SMOTE to balance the classes
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Train the logistic regression model with class weights
model = LogisticRegression(max_iter=1000, class_weight='balanced')
model.fit(X_train_smote, y_train_smote)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Confusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", class_report)


Confusion Matrix:
 [[210   1]
 [ 16   0]]

Classification Report:
               precision    recall  f1-score   support

           0       0.93      1.00      0.96       211
           1       0.00      0.00      0.00        16

    accuracy                           0.93       227
   macro avg       0.46      0.50      0.48       227
weighted avg       0.86      0.93      0.89       227



# Manually inputting dummies

In [6]:
# Identify non-numeric columns
non_numeric_columns = X.select_dtypes(include=['object']).columns

# One-hot encode categorical columns
X = pd.get_dummies(X, columns=non_numeric_columns, drop_first=True)


## Reimplimenting SMOTE

In [8]:
# Load the dataset
file_path = 'Resources/US_Accidents_2022_Phoenix_Metro2.csv'
df = pd.read_csv(file_path, low_memory=False)

# Drop columns with too many missing values
df.drop(columns=['End_Lat', 'End_Lng'], inplace=True)

# Handle missing values in other columns
df.dropna(inplace=True)

# Convert date columns to datetime and then to numerical values (e.g., ordinal)
df['Accident_Date'] = pd.to_datetime(df['Accident_Date']).map(pd.Timestamp.toordinal)

# Combine classes into binary categories
df['binary_target'] = df['Severity'].apply(lambda x: 0 if x in [1, 2] else 1)

# One-hot encode categorical columns
df_encoded = pd.get_dummies(df, drop_first=True)

# Compute correlation between features and target variable
correlation = df_encoded.corr()['binary_target'].abs().sort_values(ascending=False)

# Display correlation values
print(correlation)


binary_target                                                               1.000000
Severity                                                                    0.886905
Description_Lane blocked due to crash on I-17 Southbound at AZ-101-LOOP.    0.185686
Street_N Black Canyon Hwy                                                   0.181479
Street_I-10 W                                                               0.172292
                                                                              ...   
Weather_Condition_Mostly Cloudy                                             0.004044
Humidity(%)                                                                 0.001382
Wind_Direction_VAR                                                          0.001158
Bump                                                                             NaN
Traffic_Calming                                                                  NaN
Name: binary_target, Length: 2337, dtype: float64


# Ensuring SMOTE was properly integrated

In [10]:
print("Before SMOTE:")
print(y_train.value_counts())
print("After SMOTE:")
print(pd.Series(y_train_smote).value_counts())


Before SMOTE:
binary_target
0    490
1     38
Name: count, dtype: int64
After SMOTE:
binary_target
0    490
1    490
Name: count, dtype: int64


# Changing approach to RandomForest
## Seeing that linear regression was too simple for the data, a different approach seemed neccesary.
### Based on Bagging (A process used in statistics) we create "Trees" to create multiple regression tasks

In [12]:
# Load the dataset
file_path = 'Resources/US_Accidents_2022_Phoenix_Metro2.csv'
df = pd.read_csv(file_path, low_memory=False)

# Drop columns with too many missing values
df.drop(columns=['End_Lat', 'End_Lng'], inplace=True)

# Handle missing values in other columns
df.dropna(inplace=True)

# Convert date columns to datetime and then to numerical values (e.g., ordinal)
df['Accident_Date'] = pd.to_datetime(df['Accident_Date']).map(pd.Timestamp.toordinal)

# Combine classes into binary categories
df['binary_target'] = df['Severity'].apply(lambda x: 0 if x in [1, 2] else 1)

# Create labels and features
y = df['binary_target']
X = df.drop(columns=['Severity', 'binary_target'])

# Identify non-numeric columns
non_numeric_columns = X.select_dtypes(include=['object']).columns

# One-hot encode categorical columns
X = pd.get_dummies(X, columns=non_numeric_columns, drop_first=True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Apply SMOTE to balance the classes
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Feature selection using RandomForest
selector = SelectFromModel(RandomForestClassifier(n_estimators=100, random_state=42))
selector.fit(X_train_smote, y_train_smote)
X_train_smote_selected = selector.transform(X_train_smote)
X_test_selected = selector.transform(X_test)

# Train the RandomForest classifier
model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
model.fit(X_train_smote_selected, y_train_smote)

# Predict on the test set
y_pred = model.predict(X_test_selected)

# Evaluate the model
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Confusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", class_report)

# Additional evaluation: Precision-Recall AUC
y_pred_prob = model.predict_proba(X_test_selected)[:, 1]
precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)
pr_auc = auc(recall, precision)

print("Precision-Recall AUC: ", pr_auc)


Confusion Matrix:
 [[209   2]
 [ 15   1]]

Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.99      0.96       211
           1       0.33      0.06      0.11        16

    accuracy                           0.93       227
   macro avg       0.63      0.53      0.53       227
weighted avg       0.89      0.93      0.90       227

Precision-Recall AUC:  0.374246396687161


# Introducing Gradient Boosting or XGBoost to rebalance the classes

In [14]:
# Load the dataset
file_path = 'Resources/US_Accidents_2022_Phoenix_Metro2.csv'
df = pd.read_csv(file_path, low_memory=False)

# Drop columns with too many missing values
df.drop(columns=['End_Lat', 'End_Lng'], inplace=True)

# Handle missing values in other columns
df.dropna(inplace=True)

# Convert date columns to datetime and then to numerical values (e.g., ordinal)
df['Accident_Date'] = pd.to_datetime(df['Accident_Date']).map(pd.Timestamp.toordinal)

# Combine classes into binary categories
df['binary_target'] = df['Severity'].apply(lambda x: 0 if x in [1, 2] else 1)

# Create labels and features
y = df['binary_target']
X = df.drop(columns=['Severity', 'binary_target'])

# Identify non-numeric columns
non_numeric_columns = X.select_dtypes(include=['object']).columns

# One-hot encode categorical columns
X = pd.get_dummies(X, columns=non_numeric_columns, drop_first=True)

# Clean feature names to remove problematic characters
X.columns = [re.sub(r'[<>\[\]]', '_', col) for col in X.columns]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Apply SMOTE to balance the classes
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Feature selection using XGBoost
selector = SelectFromModel(xgb.XGBClassifier(n_estimators=100, random_state=42))
selector.fit(X_train_smote, y_train_smote)
X_train_smote_selected = selector.transform(X_train_smote)
X_test_selected = selector.transform(X_test)

# Train the XGBoost classifier
model = xgb.XGBClassifier(n_estimators=100, random_state=42, scale_pos_weight=len(y_train_smote)/sum(y_train_smote))
model.fit(X_train_smote_selected, y_train_smote)

# Predict on the test set
y_pred = model.predict(X_test_selected)

# Evaluate the model
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Confusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", class_report)

# Additional evaluation: Precision-Recall AUC
y_pred_prob = model.predict_proba(X_test_selected)[:, 1]
precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)
pr_auc = auc(recall, precision)

print("Precision-Recall AUC: ", pr_auc)


Confusion Matrix:
 [[201  10]
 [  6  10]]

Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.95      0.96       211
           1       0.50      0.62      0.56        16

    accuracy                           0.93       227
   macro avg       0.74      0.79      0.76       227
weighted avg       0.94      0.93      0.93       227

Precision-Recall AUC:  0.3428297628362646


# Calculating the scale_pos_weight

In [16]:
# Calculate the scale_pos_weight parameter
counter = Counter(y_train_smote)
scale_pos_weight = counter[0] / counter[1]

# Train the XGBoost classifier with adjusted class weights
model = xgb.XGBClassifier(n_estimators=100, random_state=42, scale_pos_weight=scale_pos_weight)
model.fit(X_train_smote_selected, y_train_smote)

# Predict on the test set
y_pred = model.predict(X_test_selected)

# Evaluate the model
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Confusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", class_report)


Confusion Matrix:
 [[203   8]
 [  7   9]]

Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.96      0.96       211
           1       0.53      0.56      0.55        16

    accuracy                           0.93       227
   macro avg       0.75      0.76      0.75       227
weighted avg       0.94      0.93      0.93       227



# Implementing Hyperparameter tuning forfinding optimal parameters for the XGBoost

In [18]:
# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0]
}

# Perform grid search
grid_search = GridSearchCV(xgb.XGBClassifier(random_state=42, scale_pos_weight=scale_pos_weight), param_grid, cv=5, scoring='f1')
grid_search.fit(X_train_smote_selected, y_train_smote)

# Get the best model from grid search
best_model = grid_search.best_estimator_

# Predict on the test set
y_pred = best_model.predict(X_test_selected)

# Evaluate the best model
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Best Parameters:\n", grid_search.best_params_)
print("Confusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", class_report)


Best Parameters:
 {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'subsample': 1.0}
Confusion Matrix:
 [[203   8]
 [  7   9]]

Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.96      0.96       211
           1       0.53      0.56      0.55        16

    accuracy                           0.93       227
   macro avg       0.75      0.76      0.75       227
weighted avg       0.94      0.93      0.93       227



# Threshold adjustments to optimize the F1-score

In [20]:
# Predict probabilities
y_pred_prob = best_model.predict_proba(X_test_selected)[:, 1]

# Calculate precision-recall curve
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_prob)
f1_scores = 2 * (precision * recall) / (precision + recall)

# Find the best threshold
best_threshold = thresholds[np.argmax(f1_scores)]

# Adjust predictions based on the best threshold
y_pred_adjusted = (y_pred_prob >= best_threshold).astype(int)

# Evaluate the adjusted predictions
conf_matrix_adjusted = confusion_matrix(y_test, y_pred_adjusted)
class_report_adjusted = classification_report(y_test, y_pred_adjusted)

print("Best threshold:", best_threshold)
print("Confusion Matrix with Adjusted Threshold:\n", conf_matrix_adjusted)
print("\nClassification Report with Adjusted Threshold:\n", class_report_adjusted)


Best threshold: 0.7145883
Confusion Matrix with Adjusted Threshold:
 [[205   6]
 [  7   9]]

Classification Report with Adjusted Threshold:
               precision    recall  f1-score   support

           0       0.97      0.97      0.97       211
           1       0.60      0.56      0.58        16

    accuracy                           0.94       227
   macro avg       0.78      0.77      0.77       227
weighted avg       0.94      0.94      0.94       227

